In [1]:
from camel.societies import RolePlaying
from camel.types import TaskType, ModelType, ModelPlatformType
from camel.models import ModelFactory
import os
from camel.configs import ZhipuAIConfig
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())

# 设置代理
#os.environ["http_proxy"] = "http://127.0.0.1:7897"
#os.environ["https_proxy"] = "http://127.0.0.1:7897"

# model = ModelFactory.create(
#     model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
#     model_type="Qwen/Qwen2.5-72B-Instruct",
#     url='https://api-inference.modelscope.cn/v1/',
#     api_key=os.getenv('ModelFactory_Key')
# )
model = ModelFactory.create(
    model_platform=ModelPlatformType.ZHIPU,
    model_type=ModelType.GLM_4,
    model_config_dict=ZhipuAIConfig(temperature=0.2).as_dict(),
    api_key=os.environ.get("ZHIPUAI_API_KEY"),
    url=os.environ.get("ZHIPUAI_API_BASE_URL"),
)

In [3]:
# from camel.agents import ChatAgent

# agent = ChatAgent(model=model)
# response = agent.step("你是谁?")
# response

In [8]:
#构建第一个agent-society

#1.定义指令参数
task_kwargs = {
    'task_prompt': '制定一个计划去过去并进行改变。',
    'with_task_specify': True,#开启后，将会有一个agent将我们的初始prompt进一步明确化
    'task_specify_agent_kwargs': {'model': model}
}
#2.指令发送者参数
user_role_kwargs = {
    'user_role_name': '一个雄心勃勃的渴望成为时间旅行者的人',
    'user_agent_kwargs': {'model': model}
}
#3.指令接收者参数
assistant_role_kwargs = {
    'assistant_role_name': '最优秀的实验物理学家',
    'assistant_agent_kwargs': {'model': model}
}

society = RolePlaying(
    **task_kwargs,             # 任务参数
    **user_role_kwargs,        # 指令发送者的参数
    **assistant_role_kwargs,   # 指令接收者的参数
    critic_role_name='human',
    with_critic_in_the_loop=True,
    output_language="中文",
)

In [9]:
#定义终止函数
def is_terminated(response):
    """
    判断响应结果是否终止
    """
    if response.terminated:
        role = response.msg.role_type.name()
        reason = response.info['termination_reasons']
        print(f"[{role}] 终止对话，原因: {reason}")
    return response.terminated

In [10]:
society.init_chat()

BaseMessage(role_name='最优秀的实验物理学家', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict=None, content='Now start to give me instructions one by one. Only reply with Instruction and Input.', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)

In [11]:
#开始对话
def run(society, round_limit: int=3):
    #获取AI助手到AI用户的初始消息
    input_msg = society.init_chat()

    #开始对话
    for _ in range(round_limit):
        #获取这一轮的两个响应
        assistant_response, user_response = society.step(input_msg)
        
        #检查终止条件
        if is_terminated(assistant_response) or is_terminated(user_response):
            break
        # 获取结果
        print(f'[AI用户]: {user_response.msg.content}.\n')
        #检查任务是否结束
        if 'CAMEL_TASK_DONE' in user_response.msg.content:
            print("任务完成！")
            break
        print(f'[AI助手]: {assistant_response.msg.content}.\n')

        #获取下一轮的输入消息
        input_msg = assistant_response.msg
    return None

run(society, round_limit=5)


> Proposals from 一个雄心勃勃的渴望成为时间旅行者的人 (RoleType.USER). Please choose an option:
Option 1:
Instruction: 确定公元1361年朱元璋决战的具体时间和地点。
Input: None
Option 2:
Input by Kill Switch Engineer.
Option 3:
Stop!!!


2025-09-01 07:40:18,270 - camel.models.model_manager - ERROR - Error processing with model: <camel.models.zhipuai_model.ZhipuAIModel object at 0x73870a2f9f60>
2025-09-01 07:40:18,271 - camel.agents.chat_agent - ERROR - An error occurred while running model glm-4, index: 0
Traceback (most recent call last):
  File "/workspaces/handy-multi-agent/camel/camel/agents/chat_agent.py", line 777, in _get_model_response
    response = self.model_backend.run(
  File "/workspaces/handy-multi-agent/camel/camel/models/model_manager.py", line 226, in run
    raise exc
  File "/workspaces/handy-multi-agent/camel/camel/models/model_manager.py", line 216, in run
    response = self.current_model.run(messages, response_format, tools)
  File "/workspaces/handy-multi-agent/camel/camel/models/base_model.py", lin

ModelProcessingError: Unable to process messages: the only provided model did not run successfully. Error: Error code: 400 - {'error': {'code': '1213', 'message': '未正常接收到prompt参数。'}}